In [34]:
import dspy

lm = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [52]:
scorer1 = dspy.Predict(signature="statement, personality, question-> impact_score, relevance_score")
scorer2 = dspy.ChainOfThought(signature="statement, personality, question-> impact_score, relevance_score")
# scorer3 = dspy.ProgramOfThought(signature="statement, personality, question-> impact_score")
scorer4 = dspy.ReAct(signature="statement, personality, question-> impact_score, relevance_score", tools=[])
scorer5 = dspy.MultiChainComparison(signature="statement, personality, question-> impact_score, relevance_score")

In [53]:
question = "You are given a statement and a personality. Determine the impact and relevance of the statement based on the personality on a scale from 0 to 1 with 0 meaning no relevance and 1 being highly relevant. The impact score shows how much the statement affects the personality. The relevance score shows how much the statement is relevant or close to the personality."
statement = "Bob did not help his friends with their homework."
personality = "Bob is a helpful person. He is always willing to lend a hand to his friends."

In [54]:
answer1 = scorer1(question=question, statement=statement, personality=personality)
answer2 = scorer2(question=question, statement=statement, personality=personality)
# answer3 = scorer3(question=question, statement=statement, personality=personality)
answer4 = scorer4(question=question, statement=statement, personality=personality)

In [41]:
answer5 = scorer5(question=question, statement=statement, personality=personality, completions=[])

AssertionError: The number of attempts (0) doesn't match the expected number M (3). Please set the correct value for M when initializing MultiChainComparison.

In [55]:
print("PREDICT: ", answer1)
print("CHAIN OF THOUGHT: ", answer2)
# print("PROGRAM OF THOUGHT: ", answer3)
print("REACT: ", answer4)
# print("MULTI CHAIN COMPARISON: ", answer5)

PREDICT:  Prediction(
    impact_score='0.6',
    relevance_score='0.4'
)
CHAIN OF THOUGHT:  Prediction(
    reasoning='The statement "Bob did not help his friends with their homework" contradicts Bob\'s personality of being a helpful person. This means that the impact score will be low, as it goes against what is expected of him. The relevance score will also be low because the action described in the statement does not align with his typical behavior.',
    impact_score='0.2',
    relevance_score='0.3'
)
REACT:  Prediction(
    trajectory={'thought_0': "The statement seems to contradict Bob's personality.", 'tool_name_0': 'finish', 'tool_args_0': {}, 'observation_0': 'Completed.'},
    reasoning="The statement contradicts Bob's personality, indicating a low impact score since it goes against his helpful nature. The relevance score is high because the statement directly relates to Bob's character.",
    impact_score='0.2',
    relevance_score='0.8'
)


In [56]:
dspy.inspect_history(n=6)





[2025-01-09T18:36:31.775191]

System message:

Your input fields are:
1. `statement` (str)
2. `personality` (str)
3. `question` (str)
4. `trajectory` (str)

Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (typing.Literal[finish])
3. `next_tool_args` (dict[str, typing.Any])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## statement ## ]]
{statement}

[[ ## personality ## ]]
{personality}

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must be one of: finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "object"}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `statement`, `personality`, `question`, produce the fi

In [58]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Example completions generated by a model for reference
completions = [
    dspy.Prediction(rationale="I recall that during clear days, the sky often appears this color.", answer="blue"),
    dspy.Prediction(rationale="Based on common knowledge, I believe the sky is typically seen as this color.", answer="green"),
    dspy.Prediction(rationale="From images and depictions in media, the sky is frequently represented with this hue.", answer="blue"),
]

# Pass signature to MultiChainComparison module
compare_answers = dspy.MultiChainComparison(BasicQA)

# Call the MultiChainComparison on the completions
question = 'What is the color of the sky?'
final_pred = compare_answers(completions, question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after comparison): {final_pred.answer}")
print(f"Final Rationale: {final_pred.rationale}")
print(final_pred)

Question: What is the color of the sky?
Final Predicted Answer (after comparison): Blue
Final Rationale: The color of the sky is often blue due to a phenomenon called Rayleigh scattering, where shorter wavelengths of light (like blue and violet) are scattered more than longer wavelengths (like red and orange), giving the sky its distinctive hue.
Prediction(
    rationale='The color of the sky is often blue due to a phenomenon called Rayleigh scattering, where shorter wavelengths of light (like blue and violet) are scattered more than longer wavelengths (like red and orange), giving the sky its distinctive hue.',
    answer='Blue'
)


In [73]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(format="integfwefewfer", desc="return a JSON object with the answer")
    
model = dspy.ChainOfThought(BasicQA)

x = model(question="What is 2 + 2?")

print(x)

Prediction(
    reasoning='The calculation of 2 + 2 is a basic arithmetic operation that involves adding two numbers together. In this case, we are simply combining the values of 2 and 2 to get a total.',
    answer='{"answer": "4"}'
)


In [74]:
dspy.inspect_history()





[2025-01-09T18:59:06.149303]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str): return a JSON object with the answer

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

[[ ## question ## ]]
What is 2 + 2?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The calculation of 2 + 2 is a basic arithmetic operation that involves adding two numbers together. In this case, we are simply combining the values of 2 and 2 to get a total.

[[ ## answer ## ]]
{"answer": "4